In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from pyproj import Transformer


df = main.unique(['lat','long'])
coords = df.select(['lat','long']).to_numpy()
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
projected_coords = np.array([transformer.transform(lon, lat) for lat, lon in coords])
scale = StandardScaler().fit(projected_coords)
projected_coords = scale.transform(projected_coords)


df2 = test.unique(['lat','long'])
coords2 = df2.select(['lat','long']).to_numpy()
projected_coords2 = np.array([transformer.transform(lon, lat) for lat, lon in coords2])
projected_coords2 = scale.transform(projected_coords2)


KNN

In [ ]:
from sklearn.cluster import KMeans
import polars as pl

kmeans = KMeans(n_clusters=150, random_state=42)  
kmeans.fit(projected_coords)

labels = kmeans.labels_
df = df.with_columns(pl.Series("cluster", labels))

labels2 = kmeans.predict(projected_coords2)
df2 = df2.with_columns(pl.Series("cluster", labels2))

DBSCAN

In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=5,
    min_samples=3,
    metric='euclidean',
    prediction_data=True
).fit(projected_coords)

labels = clusterer.labels_
df = df.with_columns(pl.Series("cluster", labels))


labels2, strengths = hdbscan.approximate_predict(clusterer, projected_coords2)
df2 = df2.with_columns(pl.Series("cluster", labels2))

Visualization

In [ ]:
chart = alt.Chart(df).mark_circle(size=10).encode(
    longitude='long:Q',
    latitude='lat:Q',
    color='cluster:N',
    tooltip=['cluster']
).project(
    type='mercator'
).properties(
    width=600,
    height=400,
).interactive()
chart

In [ ]:
main = main.join(df.select(['lat', 'long', 'cluster']), 
on=['lat', 'long'], how='left').select(pl.exclude(['아파트명','지역']))
test = test.join(df2.select(['lat', 'long', 'cluster']), 
on=['lat', 'long'], how='left').select(pl.exclude(['아파트명','지역']))